In [1]:
#Importar los módulos necesarios para el funcionamiento de nuestro programa
import requests
import pandas as pd

In [2]:
#cURL(cmd) Request del website convertida a Python utilizando curl.trillworks.com
#### Esta request busca autos 'Tesla' en 'San Francisco, CA' ###
headers = {
    'authority': 'www.autolist.com',
    'sec-ch-ua': '^\\^Google',
    'dnt': '1',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'accept': '*/*',
    'x-requested-with': 'XMLHttpRequest',
    'x-autolist-session-guid': '256dd1a2-3af4-4fd1-9625-f33a62837975',
    'sec-ch-ua-platform': '^\\^Windows^\\^',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.autolist.com/listings',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'ec=eyJlZGdlVXNlckFnZW50IjoiTW96aWxsYS81LjAgKFdpbmRvd3MgTlQgMTAuMDsgV2luNjQ7IHg2NCkgQXBwbGVXZWJLaXQvNTM3LjM2IChLSFRNTCwgbGlrZSBHZWNrbykgQ2hyb21lLzkzLjAuNDU3Ny44MiBTYWZhcmkvNTM3LjM2In0=; client_guid_timestamp=6cb7eb43-f98b-4a30-b5d8-b09d1f73972e.1631848570197; scavenger-a493=i2edJn8U8quG2BtrlZCXcua4KB62fcJRrAhaI/t9eYZNJczdHQYj3P0Y4s6myhEl4IbJl8vp9L0oN+SXDh2fFcTvJ/vA/hPyRJ0ohE6LnQGIee5ACCL3nGqW5xFg3QnijfTypiJhVgTfPjk3c81AtkxzR54e+BHqQL0DI6/oWNd7iIkgvHaWjuilZuITytvFV0IAUJ0bX041NuAIFpx8K90bzfPLeAKgkwc0lpwB3cLwm8NcNKMD3xELdjI6leANl5jZJ9KzZ3K+5J7nCpxnAynFJJt2e95bby70BTVZ0qdWtSWGgwFxEdjmesrF42H7EgAjt4LMIKDuRVtbhVwu/q/wUhPoUZvZb6F+CbFpxMXuZTpd27JmWUrboqGTfyS3Nr7gV1kl6hr+8w0aOlmo8NlrdmNRb0u51Ex4zu4JTymiAUsWvS9JhNr/uvk4GVvM3kafDQeKq7v+xHkB/lCvPkqLEifYP7VVan6h5ZmvaQTgULjs0N5JhUd76Pt6c/tY5R0v8R67gjgbvW8n/4Gqzw==',
}

In [3]:
#Crear listas vacías a las que se agregarán los datos que queremos obtener del website
modelo = []
millaje = [] #Se puede convertir en kilometraje haciendo int() y luego * 1.60934
año = []
concesionaria = []
precio = []

In [4]:
#### La diferencia entre esta request y la singular es que agrega un loop en el parámetro 'page'
#### para que la búsqueda vaya avanzando y muestre los siguientes resultados
#### motivo por el cual la mitad del santo programa tiene que estar dentro del loop y me parece un método HORRENDO
#### loco no podemos hacer page_number = input("escribí la página que quieras scrapear")
#### y luego vas a ('page', page_number)
#### no sé, digo

for i in range(1,6):
    params = (
        ('make', 'Tesla'),
        ('location', 'San^%^20Francisco,^%^20CA'),
        ('latitude', '37.7749295'),
        ('longitude', '-122.4194155'),
        ('radius', '50'),
        ('page', str(i)),
    )
    #Definir nuestras variables para que sea más fácil acceder a cada uno de sus elementos
    response = requests.get('https://www.autolist.com/api/cwv/seo/listings', headers=headers, params=params)
    resultados_json = response.json()
    search_results = resultados_json['search_results']
    
    #Crear un loop que en orden añada uno por uno los datos que queremos a las listas que definimos en el paso anterior
    for result in search_results:
        modelo.append(result['model'])
        millaje.append(result['mileage'])
        año.append(result['year']) 
        concesionaria.append(result['dealer_name']) 
        precio.append(result['price'])

In [5]:
#Crear y visualizar un dataframe para estimar si nuestros resultados son correctos
tesla_df_multiple = pd.DataFrame({'Modelo':modelo, 'Millaje':millaje, 'Año':año, 'Concesionaria':concesionaria, 'Precio':precio})
tesla_df_multiple

,Modelo,Millaje,Año,Concesionaria,Precio
0,Model 3,"30,924 Miles",2018,Shift SF - Test Drives Delivered To You,"$42,950"
1,Model 3,"14,758 Miles",2019,Shift SF - Test Drives Delivered To You,"$41,450"
2,Model 3,"7,148 Miles",2018,Shift SF - Test Drives Delivered To You,"$45,950"
3,Model 3,"26,080 Miles",2018,Shift SF - Test Drives Delivered To You,"$43,450"
4,Model 3,"18,556 Miles",2019,Shift SF - Test Drives Delivered To You,"$41,950"
...,...,...,...,...,...
95,Model Y,"2,508 Miles",2021,Future Ford Of Concord,"$65,950"
96,Model Y,"8,420 Miles",2021,Future Ford Of Concord,"$64,950"
97,Model S,"46,870 Miles",2016,Future Ford Of Concord,"$53,950"
98,Model S,"92,900 Miles",2014,Calidad Motors,"$35,998"


In [6]:
#Exportar nuestro dataframe a un archivo excel
tesla_df_multiple.to_excel('tesla_sfca_multiple.xlsx', index=False)